# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [35]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/akanksha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
# Read your api key environment variable
api_key = os.getenv("NEWS_API_KEY")

In [8]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [15]:
# Fetch the Bitcoin news articles
bitcoin_articles = newsapi.get_everything(q="bitcoin", language="en")

In [16]:
# Fetch the Ethereum news articles
ethereum_articles = newsapi.get_everything(q="ethereum", language="en")

In [38]:
# Method to get sentiments for articles 
def get_sentiments(articles):
    """
    Gets the sentiments for all the articles provided
    
    Params:
        articles- dictionary of the response from newspi call
    Returns:
        sentiments_df- Dataframe of all the titles and associated sentiment scores 
    """
    analyzer = SentimentIntensityAnalyzer()
    sentiments = []
    for article in articles['articles']:
        sentiment = analyzer.polarity_scores(article['content'])
        sentiments.append({
            'title': article['title'], 
            'compound': sentiment["compound"],
            'pos': sentiment["pos"],
            'neu': sentiment["neu"],
            'neg': sentiment["neg"]
            })
    sentiments_df = pd.DataFrame(sentiments)
    return sentiments_df

In [39]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiment_df = get_sentiments(bitcoin_articles)
bitcoin_sentiment_df.head()

,title,compound,pos,neu,neg
0,New York passes a bill to limit bitcoin mining,-0.5574,0.000,0.902,0.098
1,Miami’s Bitcoin Conference Left a Trail of Har...,0.0772,0.036,0.964,0.000
2,Why the Central African Republic adopted Bitcoin,0.5106,0.164,0.836,0.000
3,Chipotle now accepts cryptocurrency payments,-0.1027,0.000,0.960,0.040
4,Bitcoin value drops by 50% since November peak,0.3400,0.072,0.928,0.000


In [40]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiment_df = get_sentiments(ethereum_articles)
ethereum_sentiment_df.head()

,title,compound,pos,neu,neg
0,Here's what NFTs look like on Instagram,0.6486,0.135,0.865,0.000
1,GameStop Dunks Its Head Into the Crypto Kiddie...,-0.1027,0.000,0.960,0.040
2,GameStop Launching Digital Wallet for Cryptocu...,0.1280,0.046,0.954,0.000
3,New York passes a bill to limit bitcoin mining,-0.5574,0.000,0.902,0.098
4,'Pay more attention' Ethereum co-founder says ...,0.0258,0.034,0.966,0.000


In [45]:
# Describe the Bitcoin Sentiment
bitcoin_sentiment_describe = bitcoin_sentiment_df.describe()
bitcoin_sentiment_describe

,compound,pos,neu,neg
count,20.000000,20.00000,20.000000,20.000000
mean,-0.098455,0.06175,0.860850,0.077500
std,0.431188,0.06477,0.100073,0.077527
min,-0.859300,0.00000,0.557000,0.000000
25%,-0.411525,0.00000,0.837500,0.030000
50%,-0.199350,0.04900,0.881000,0.071500
75%,0.249275,0.08050,0.923000,0.099250
max,0.750600,0.20200,0.964000,0.300000


In [46]:
# Describe the Ethereum Sentiment
ethereum_sentiment_describe = ethereum_sentiment_df.describe()
ethereum_sentiment_describe

,compound,pos,neu,neg
count,20.000000,20.000000,20.000000,20.000000
mean,-0.022370,0.050150,0.900700,0.049350
std,0.431256,0.056688,0.057349,0.052294
min,-0.690800,0.000000,0.803000,0.000000
25%,-0.381800,0.000000,0.856000,0.000000
50%,-0.051350,0.040000,0.912500,0.042500
75%,0.365100,0.075250,0.954250,0.079000
max,0.690800,0.178000,1.000000,0.178000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin has the highest mean positive score of 0.061750000000000006

In [60]:
print(f'Bitcoin mean positive score: {bitcoin_sentiment_describe.loc["mean"]["pos"]}')
print(f'Ethereum mean positive score: {ethereum_sentiment_describe.loc["mean"]["pos"]}')

Bitcoin mean positive score: 0.061750000000000006
Ethereum mean positive score: 0.05014999999999999


Q: Which coin had the highest compound score?

A: Bitcoin had the highest compound score of 0.7506

In [62]:
print(f'Bitcoin max compound score: {bitcoin_sentiment_describe.loc["max"]["compound"]}')
print(f'Ethereum max compound score: {ethereum_sentiment_describe.loc["max"]["compound"]}')

Bitcoin max compound score: 0.7506
Ethereum max compound score: 0.6908


Q: Which coin had the highest positive score?

A: Bitcoin had the highest positive score of 0.202

In [64]:
print(f'Bitcoin max positive score: {bitcoin_sentiment_describe.loc["max"]["pos"]}')
print(f'Ethereum max positive score: {ethereum_sentiment_describe.loc["max"]["pos"]}')

Bitcoin max positive score: 0.202
Ethereum max positive score: 0.178


---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [11]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

# Create a list of stopwords
# YOUR CODE HERE!

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---